In [147]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment
def convert_string(input_str):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    
    # Search for the pattern in the string
    match = re.search(pattern, input_str)
    if match:
    # Extract and print the date if found
        date_str = match.group(0)
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')

        return date
    else:
        'No date found'    
#convert time string to datetime object
def convert_time_string(posted_date):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'

    # Search for the pattern in the string and extract the date
    match = re.search(pattern, posted_date)
    if match:
        date_str = match.group(0)
        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj
    else:
        return None 
def get_time_string(date_str):
    if len(date_str)<=5:
        crawl_time = datetime.fromtimestamp(time.time())
        year = str(crawl_time.year)
        date_str = date_str+"/"+year
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')
    else:
        date = convert_string(date_str)
        date_obj = datetime.strptime(date,'%Y-%m-%d')
    return date_obj, date


def remove_div(article):
    divs = article.find_all('div')
    empty_divs = [div for div in divs if not div.text.strip() and not div.contents]
    if not empty_divs:
        return  # No more empty divs, stop recursion
    for div in empty_divs:
        div.decompose()
def get_content_gamek(url):
    # def get_content_autodaily()
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('div', class_ = 'topdetail').find('h1').text.strip()
    date = soup.find('p', class_ = 'mgt15').text.strip()
    published_date = convert_string(date)
    article = soup.find('div', class_ = 'rightdetail')
    for script_or_style in article(['script', 'style','iframe']):
        script_or_style.decompose()
    source_element  = article.find('div', class_ = 'link-source-wrapper is-web clearfix mb20')
    source_element.decompose()
    related_news = article.find('div', class_ = 'VCSortableInPreviewMode link-content-footer IMSCurrentEditorEditObject')
    related_news.decompose()
    video_captions = article.find_all('div', class_ = 'VideoCMS_Caption')
    for caption in video_captions:
        caption.decompose()
    figures = article.find_all('figure',class_ = 'VCSortableInPreviewMode')
    for figure in figures:
        img = figure.find('img')
        new_img = soup.new_tag('img')
        new_img['style'] = "text-align: center;"
        new_img['src'] = img['data-original']
        new_img['alt'] = img['alt']
        img.insert_before(new_img)
        img.decompose()
    for i in article.find_all('img'):
            i['class'] = "aligncenter"
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: gamek.vn"  # Set the content of <i> tag
    article.append(source_tag)
    for i in article.find_all('div', style="display: none"):
        i.decompose()
    a_tags = article.find_all('a')
    for a_tag in a_tags:
        if len(a_tag.contents) == 1 and a_tag.find('img'):
            a_tag['href'] = 'javascript:void(0)'
        else:
            tag_text = a_tag.get_text()
            # Replace the tag with its text content
            a_tag.replace_with(tag_text)
            a_tag.text.strip()
    for i in article.find_all(recursive = True):
        if i.name not in ['figcaption', 'img']:
            try:
                del i['onclick']
                del i['id']
                del i['class']
                del i['style']
            except AttributeError:
                continue
            except TypeError:
                continue
    tags_to_remove = article.find_all(['span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    for script_or_style in article(['script', 'style']):
        script_or_style.decompose()
    return article, title, published_date
def get_post(url):
    try:
        content,title,published_date = get_content_gamek(url) 
        return content,title,published_date
    except AttributeError as e:
        print(e)
    
def get_list_url(cate_url):
    response = requests.get(cate_url)
    time.sleep(3)
    soup = BeautifulSoup(response.content, 'html.parser')
    urls = []
    try:
        link_element = soup.find('div', class_ = 'newsupdate mgt0')
        list = link_element.find_all('li', class_ = 'top')
        for i in list:
            path = i.find('a')['href']
            url = 'https://gamek.vn' + path
            urls.append(url)
    except AttributeError:
        link_element = soup.find('div', class_ = 'list_left').find_all('ul')
        link_element[1].decompose()
        for ul in link_element:
            list_url = ul.find_all('li')
            for li in list_url:
                path = li.find('a')['href']
                url = 'https://gamek.vn' + path
                urls.append(url)
    return urls
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time()-4*24*3600)
    for i in range(0,len(urls)):
        try:
            published_date = get_content_gamek(urls[i])[2]
            date_posted_norm = datetime.strptime(published_date, '%Y-%m-%d')
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(urls[i])
                #print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls
#add list url to json
#add list url to json
def add_list(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):  
            urls = get_list_url(web_json_obj['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_json_obj['urls'][i]['sub-category'][j]['url'])
            web_json_obj['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)
# add post content from get content function to json object
def add_post(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):
            web_json_obj['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_json_obj['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_json_obj['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_json_obj['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_json_obj['urls'][i]['sub-category'][j]['cate_id'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
#add all necessary information to json object
def get_news_gamek():
    _gamek= {
            "home_page":"https://gamek.vn/",
            "urls":{
                "Esport":
                {
                 "url":"https://gamek.vn/#",
                 "sub-category":{  
                    0:{"name":"LMHT",
                    "url":"https://gamek.vn/lien-minh-huyen-thoai.htm",
                    "cate_id":61,
                    "url_list" : []},
                    1:{"name":"Liên quân Mobile",
                    "url":"https://gamek.vn/lien-quan-mobile.htm",
                    "cate_id":61,
                    "url_list" : []},
                    2:{"name":"LMHT Tốc chiến",
                    "url":"https://gamek.vn/toc-chien.htm",
                    "cate_id":61,
                    "url_list" : []},
                    3:{"name":"Game Online",
                    "url":"https://gamek.vn/game-online.chn",
                    "cate_id":61,
                    "url_list" : []},
                    4:{"name":"PC/Console",
                    "url":"https://gamek.vn/pc-console.chn",
                    "cate_id":61,
                    "url_list" : []}
                }
            }
        }
    }
#
    add_list(_gamek)
    add_post(_gamek)
    return _gamek
#send post content to wordpress via endpoint
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": 'draftpost',#'5goalvodichcmnl',  
        # Replace with your actual access token
        "published_date": published_date,
        "domain":"eva"
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    _gamek = get_news_gamek()
    for i in list(_gamek['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(_gamek['urls'][i]['sub-category'].keys()):
            url_list =  _gamek['urls'][i]['sub-category'][j]['url_list']
            print(url_list)
            for t in range(0,len(url_list)):
                content = _gamek['urls'][i]['sub-category'][j]['content'][t]['text']
                title = _gamek['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = _gamek['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = _gamek['urls'][i]['sub-category'][j]['cate_id']
                print(title, url_list[t])
                #send_post_to_5goals(title,str(content), cate_id, published_date)
                try:
                    text_len = len(content.text)
                    if text_len <450:
                        print(content.text)
                        continue
                    else:
                        send_post_to_5goals(title,str(content),cate_id,published_date)
                        print("Good job, well done!!!")
                except (AttributeError,TypeError):
                    continue
if __name__ == '__main__':
    main()

In [148]:
main()

Esport 0 https://gamek.vn/lien-minh-huyen-thoai.htm
Esport 1 https://gamek.vn/lien-quan-mobile.htm
Esport 2 https://gamek.vn/toc-chien.htm
Esport 3 https://gamek.vn/game-online.chn
Esport 4 https://gamek.vn/pc-console.chn
'NoneType' object has no attribute 'decompose'
Esport 3 61 Game Online Game Online Những tựa game đáng chú ý nhất khép lại năm 2023, ít siêu phẩm nhưng chất lượng thì đáng nể https://gamek.vn/nhung-tua-game-dang-chu-y-nhat-khep-lai-nam-2023-it-sieu-pham-nhung-chat-luong-thi-dang-ne-178231207144532414.chn
Esport 3 61 Game Online Game Online Cha đẻ PUBG tung tiền tấn để cứu vớt siêu phẩm, tránh phải đóng cửa chỉ sau một năm https://gamek.vn/cha-de-pubg-tung-tien-tan-de-cuu-vot-sieu-pham-tranh-phai-dong-cua-chi-sau-mot-nam-178231206153106246.chn
Esport 3 61 Game Online Game Online Nạp cả trăm triệu vào game, người chơi hoảng hốt khi NPH "bỏ của chạy lấy người" https://gamek.vn/nap-ca-tram-trieu-vao-game-nguoi-choi-hoang-hot-khi-nph-bo-cua-chay-lay-nguoi-17823120611312069

In [149]:
get_content_gamek('https://gamek.vn/nhung-ly-do-khien-contra-tuong-dai-mot-thoi-nay-chi-con-la-hoai-niem-voi-game-thu-178231205141837042.chn')

(<div class="rightdetail"><h2 data-field="sapo" data-role="sapo">Tựa game tuổi thơ của không ít game thủ cũng có câu chuyện hành trình đầy uẩn khúc và đặc biệt.</h2><zone></zone><div data-check-position="gamek_detail_position_start"></div><div data-field="body" data-role="content"><p>Đối với thế hệ 8-9x, đặc biệt là những người đã từng trải qua quãng thời gian "chinh chiến" đầy máu lửa với các tựa game của điện tử 4 nút, Contra có lẽ là cái tên thường xuyên được nhắc tới nhất. Cũng dễ hiểu khi ở thời điểm mới ra mắt, Contra nhanh chóng được coi là một trong những tựa game hành động hạng A với sự đơn giản nhưng lại hợp lý tới kỳ lạ. Thành công vang dội của Contra thậm chí còn giúp cho Konami vươn mình trở thành một trong những nhà phát hành game nổi tiếng nhất trên toàn cầu.</p><figure type="Photo"><img alt="photo-1701760680292" class="aligncenter" src="https://gamek.mediacdn.vn/133514250583805952/2023/12/5/photo-1701760680292-17017606803831356505377.png" style="text-align: center;"/><f

In [146]:
def get_list_url(cate_url):
    response = requests.get(cate_url)
    time.sleep(3)
    soup = BeautifulSoup(response.content, 'html.parser')
    urls = []
    try:
        link_element = soup.find('div', class_ = 'newsupdate mgt0')
        list = link_element.find_all('li', class_ = 'top')
        for i in list:
            path = i.find('a')['href']
            url = 'https://gamek.vn' + path
            urls.append(url)
    except AttributeError:
        link_element = soup.find('div', class_ = 'list_left').find_all('ul')
        link_element[1].decompose()
        for ul in link_element:
            list_url = ul.find_all('li')
            for li in list_url:
                path = li.find('a')['href']
                url = 'https://gamek.vn' + path
                urls.append(url)
    return urls


In [145]:
get_list_url('https://gamek.vn/game-nft.htm')

https://gamek.vn/ni-no-kuni-cross-worlds-tu-canh-tranh-voi-genshin-impact-toi-ke-bi-game-thu-toan-cau-che-bai-vi-tinh-nang-nft-202205311903256.chn
https://gamek.vn/nguoi-choi-viet-nam-cay-game-nft-nhieu-thu-5-the-gioi-20220607123237646.chn
https://gamek.vn/tua-game-indie-poppy-playtime-vuong-vao-chi-trich-khi-phat-hanh-phien-ban-nfts-20220524230543848.chn
https://gamek.vn/nyan-heroes-game-nft-dau-tien-cho-phep-ban-tham-gia-vao-dau-truong-battle-royale-20220522211704352.chn
https://gamek.vn/viet-nam-chinh-thuc-co-hiep-hoi-blockchain-hua-hen-lam-cau-noi-dua-nen-kinh-te-so-viet-ra-voi-the-gioi-20220517173553054.chn
https://gamek.vn/overleague-tua-game-dua-xe-blockchain-hoan-toan-mien-phi-voi-cot-truyen-va-gameplay-doc-dao-ra-mat-20220516120316619.chn
https://gamek.vn/tatsumeeko-game-nft-nhap-vai-cuc-hot-thu-hut-ca-tram-nghin-game-thu-20220514224645565.chn
https://gamek.vn/moi-sieu-dev-cua-nhieu-bom-tan-ve-lam-viec-sieu-pham-game-nft-quyet-tam-choi-lon-hua-hen-la-game-aaa-bung-no-202205101

[]

In [150]:
def get_content_gamek(url):
    # def get_content_autodaily()
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('div', class_ = 'topdetail').find('h1').text.strip()
    date = soup.find('p', class_ = 'mgt15').text.strip()
    published_date = convert_string(date)
    article = soup.find('div', class_ = 'rightdetail')
    for script_or_style in article(['script', 'style','iframe']):
        script_or_style.decompose()
    source_element  = article.find('div', class_ = 'link-source-wrapper is-web clearfix mb20')
    source_element.decompose()
    related_news = article.find('div', class_ = 'VCSortableInPreviewMode link-content-footer IMSCurrentEditorEditObject')
    related_news.decompose()
    video_captions = article.find_all('div', class_ = 'VideoCMS_Caption')
    for caption in video_captions:
        caption.decompose()
    figures = article.find_all('figure',class_ = 'VCSortableInPreviewMode')
    for figure in figures:
        img = figure.find('img')
        new_img = soup.new_tag('img')
        new_img['style'] = "text-align: center;"
        new_img['src'] = img['data-original']
        new_img['alt'] = img['alt']
        img.insert_before(new_img)
        img.decompose()
    for i in article.find_all('img'):
            i['class'] = "aligncenter"
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: gamek.vn"  # Set the content of <i> tag
    article.append(source_tag)
    for i in article.find_all('div', style="display: none"):
        i.decompose()
    a_tags = article.find_all('a')
    for a_tag in a_tags:
        if len(a_tag.contents) == 1 and a_tag.find('img'):
            a_tag['href'] = 'javascript:void(0)'
        else:
            tag_text = a_tag.get_text()
            # Replace the tag with its text content
            a_tag.replace_with(tag_text)
            a_tag.text.strip()
    for i in article.find_all(recursive = True):
        if i.name != 'figcaption':
            try:
                del i['onclick']
                del i['id']
                del i['class']
                del i['style']
            except AttributeError:
                continue
            except TypeError:
                continue
    tags_to_remove = article.find_all(['span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    for script_or_style in article(['script', 'style']):
        script_or_style.decompose()
    return article, title, published_date

In [151]:
get_content_gamek('https://gamek.vn/thi-dau-giao-huu-zeus-van-co-man-check-var-lo-lieu-the-nay-day-178231210023158871.chn')[0]

<div class="rightdetail"><h2 data-field="sapo" data-role="sapo">Dàn sao T1 luôn biết cách làm content mỗi khi có cơ hội.</h2><zone></zone><div data-check-position="gamek_detail_position_start"></div><div data-field="body" data-role="content"><p>Như đã biết, vừa qua, T1 đã phải tham dự một sự kiện có thể gọi là fan meeting do nhãn hàng RedBull - nhà tài trợ của T1, tổ chức tại Đức. Với tính chất một giải đấu showmatch, tất nhiên kết quả không thực sự quá quan trọng. Hơn nữa, các tuyển thủ T1 chắc chắn cũng sẽ nhân cơ hội này để tranh thủ du lịch nghỉ dưỡng sau một mùa giải căng thẳng cũng như chuẩn bị tinh thần và tâm lý cho năm 2024 được dự đoán là vô cùng cam go.</p><figure type="Photo"><div><img alt="T1 tham dự sự kiện do RedBull tổ chức tại Đức" src="https://gamek.mediacdn.vn/133514250583805952/2023/12/10/zeus-t1-checkvar-1-17021942385711439044791.jpg"/></div><figcaption class="PhotoCMS_Caption"><p data-placeholder="Nhập chú thích ảnh">T1 tham dự sự kiện do RedBull tổ chức tại Đức</

In [138]:
response = requests.get('https://gamek.vn/lien-minh-huyen-thoai.htm')
soup = BeautifulSoup(response.content, 'html.parser')
link_element = soup.find('div', class_ = 'list_left').find_all('ul')
link_element[1].decompose()
for ul in link_element:
    list_url = ul.find_all('li')
    for li in list_url:
        path = li.find('a')['href']
        url = 'https://gamek.vn' + path
        print(url)
    
    

https://gamek.vn/hinh-anh-dan-sao-t1-tai-giai-showmatch-khien-fan-xot-xa-chi-trich-bld-doi-vi-lich-chay-show-hanh-xac-178231210184031603.chn
https://gamek.vn/thi-dau-giao-huu-zeus-van-co-man-check-var-lo-lieu-the-nay-day-178231210023158871.chn
https://gamek.vn/nghich-ly-lmht-faker-co-nhieu-skin-hon-ca-nhung-vi-tuong-rat-duoc-yeu-thich-17823121001505082.chn
https://gamek.vn/mua-giai-moi-chua-bat-dau-fan-t1-da-nhan-tin-xau-lien-quan-faker-178231209162701709.chn
https://gamek.vn/sao-tre-cua-vcs-tung-thi-dau-tai-lck-tiep-tuc-mat-hut-blv-hoang-luan-chot-nhan-xet-phu-nhung-that-17823120900070987.chn
https://gamek.vn/gumayusi-da-khang-dinh-vai-tro-cuc-ky-quan-trong-tai-t1-chi-bang-mot-cau-noi-hua-hen-ke-vi-faker-178231209001143432.chn
https://gamek.vn/blv-hoang-luan-nhan-dinh-mot-sieu-sao-t1-co-the-dat-giai-player-of-the-year-2023-khong-phai-faker-178231208144122684.chn
https://gamek.vn/vo-dich-cktg-2023-fan-phat-hien-t1-lap-luon-thanh-tich-kho-do-178231208173052453.chn


In [131]:
link_element[0]

KeyError: 0

In [133]:
soup.find('div', class_ = 'list_left')

<div class="list_left"><ul data-marked-zoneid="gamek_game_b3"><li><a href="/hinh-anh-dan-sao-t1-tai-giai-showmatch-khien-fan-xot-xa-chi-trich-bld-doi-vi-lich-chay-show-hanh-xac-178231210184031603.chn" title='Hình ảnh dàn sao T1 tại giải showmatch khiến fan xót xa, chỉ trích BLĐ đội vì lịch chạy show "hành xác"'> <img alt='Hình ảnh dàn sao T1 tại giải showmatch khiến fan xót xa, chỉ trích BLĐ đội vì lịch chạy show "hành xác"' src="https://gamek.mediacdn.vn/zoom/252_142/133514250583805952/2023/12/10/t1-tuyenthu-6-170220819827222370936-0-0-562-900-crop-1702208339328887826354.png"/> </a><div class="list_content"><a class="list_title" href="/hinh-anh-dan-sao-t1-tai-giai-showmatch-khien-fan-xot-xa-chi-trich-bld-doi-vi-lich-chay-show-hanh-xac-178231210184031603.chn" title='Hình ảnh dàn sao T1 tại giải showmatch khiến fan xót xa, chỉ trích BLĐ đội vì lịch chạy show "hành xác"'>Hình ảnh dàn sao T1 tại giải showmatch khiến fan xót xa, chỉ trích BLĐ đội vì lịch chạy show "hành xác"</a><p class="l

In [81]:
article

<div class="rightdetail"><h2 data-field="sapo" data-role="sapo">Dàn sao T1 luôn biết cách làm content mỗi khi có cơ hội.</h2><zone id="admzonek9yy7w7l"></zone><div data-check-position="gamek_detail_position_start"></div><div class="rightdetail_content detailsmallcontent" data-field="body" data-role="content"><p>Như đã biết, vừa qua, <a class="link-inline-content" data-rel="follow" href="https://gamek.vn/mua-giai-moi-chua-bat-dau-fan-t1-da-nhan-tin-xau-lien-quan-faker-178231209162701709.chn" target="_blank" title="T1">T1</a> đã phải tham dự một sự kiện có thể gọi là fan meeting do nhãn hàng RedBull - nhà tài trợ của T1, tổ chức tại Đức. Với tính chất một giải đấu showmatch, tất nhiên kết quả không thực sự quá quan trọng. Hơn nữa, các tuyển thủ T1 chắc chắn cũng sẽ nhân cơ hội này để tranh thủ du lịch nghỉ dưỡng sau một mùa giải căng thẳng cũng như chuẩn bị tinh thần và tâm lý cho năm 2024 được dự đoán là vô cùng cam go.</p><figure class="VCSortableInPreviewMode" type="Photo"><div><img a

In [56]:
article.find_all('a')

[<a class="link-inline-content" data-rel="follow" href="https://gamek.vn/mua-giai-moi-chua-bat-dau-fan-t1-da-nhan-tin-xau-lien-quan-faker-178231209162701709.chn" target="_blank" title="T1">T1</a>,
 <a class="link-inline-content" data-rel="follow" href="https://gamek.vn/zeus-chuan-bi-tahm-kench-duong-tren-nhung-khong-duoc-su-dung-hoa-ra-tung-bop-gumayusi-178231204180351368.chn" target="_blank" title="Zeus">Zeus</a>]